In [1]:
import sqlite3 as lite
from sqlalchemy import create_engine
import pandas as pd
import random
from faker import Faker
from faker.providers import internet
from faker.providers import user_agent

import datetime as DT
from datetime import timedelta
from modules.log_dhcp5 import DHCPLog
from modules.outbound_browsing.make_outbound_traffic4 import OutboundEvent
from tqdm import tqdm

faker = Faker()
faker.add_provider(internet)
faker.add_provider(user_agent)
today = DT.date.today()

In [2]:
def generateAddresses(network):
    addresses = []
    for i in range(1,65535):
        addr = build_address(network, i)
        addresses.append(addr)
    return addresses

def build_address(network, node):
    hexy = hex(node)
    pos = hexy.index('x') + 1
    hexy = hexy[pos:].zfill( 4)
    first = int(hexy[:2], 16)
    second = int(hexy[2:], 16)
    
    return network + "." + str(first).zfill(3) + "." + str(second).zfill(3)

In [3]:
# Data Setup
with open('external_hosts.txt', 'r') as f:
    endpoints = f.readlines()
    
with open('stu_external_hosts.txt', 'r') as f:
    stu_endpoints = f.readlines()

with open('tch_external_hosts.txt', 'r') as f:
    tch_endpoints = f.readlines()

stu_endpoints += endpoints
    
tch_endpoints += endpoints

my_proto_addresses = generateAddresses("192.168")

In [4]:
db_filename = r'mydb.db'
startdate = today - DT.timedelta(days=7)

engine = create_engine(r"sqlite:///{}".format(db_filename))
sql = 'SELECT * from devices'
device_df = pd.read_sql(sql, engine)

# sql = 'SELECT * from students'
# df_students = pd.read_sql(sql, engine)
# df_students = pd.DataFrame(columns={'stuid', 'fname', 'lname', 'address', 'city', 'state', 'hphone', 'dob', 'ssn', 'grade', 'schoolassignment', 'deviceassignment'})

devices = pd.DataFrame()
df_dhcp = pd.DataFrame(columns={"id", "date", "time", "description", "ip", "macaddr"})

DHCP_LOG = []
WEB_LOG = []
df_event_summary = pd.DataFrame(columns={'stime', 'etime', 'src_ip', 'user_agent', 'ownertype'})

# j = 0

# for d in range(4, 9):

            
  #         new_log_entry = DHCPLog(j, mydate, mytime, myip, mac, 'Initiated IP Lease')
  #         DHCP_LOG.append(new_log_entry.stringify())


In [13]:
my_proto_addresses = generateAddresses("192.168")

In [14]:
len(my_proto_addresses)

65534

In [22]:
j = 0
d = 8
myaddresses = my_proto_addresses.copy()
for i in range(len(device_df)):
    if random.random() < .8:
        j = j + 1
        myip = random.choice(myaddresses)
        myaddresses.remove(myip)
        dev = device_df.iloc[i]
        mac = dev['mac_addr']
        dev['ip'] = myip                                                                                    # IP Address  
        if dev['dev_type'] == 'tablet':
            mydatetime = DT.datetime(2020, 4, 20+d) + timedelta(minutes=(int(random.randrange(510, 630))))
        elif dev['dev_type'] == 'desktop':
            mydatetime = DT.datetime(2020, 4, 20+d) + timedelta(minutes=(int(random.randrange(480, 600))))
        else:
            mydatetime = DT.datetime(2020, 4, 20+d) + timedelta(minutes=(int(random.randrange(360, 600))))

        mydate = mydatetime.date()
        mytime = mydatetime.time()
        values_to_add = {
            "date": mydate, 
            "time": mytime, 
            "description": 'Initiated IP Lease', 
            "ip": myip, 
            "macaddr": mac
        }
        row_to_add = pd.Series(values_to_add, name = j)
        df_dhcp = df_dhcp.append(row_to_add)

        mydatetime2 = mydatetime + timedelta(hours=8)
        mydate2 = mydatetime2.date()
        mytime2 = mydatetime2.time()
        values_to_add = {
            "date": mydate2, 
            "time": mytime2, 
            "description": 'IP Lease Expired', 
            "ip": myip, 
            "macaddr": mac
        }
        row_to_add = pd.Series(values_to_add, name = j)
        df_dhcp = df_dhcp.append(row_to_add)

        if dev['dev_type'] == 'tablet':
            time = faker.date_time_between(start_date=mydatetime, end_date='+5h')
        elif dev['dev_type'] == 'desktop':
            time = faker.date_time_between(start_date=mydatetime, end_date='+8h')
        else:
            time = faker.date_time_between(start_date=mydatetime, end_date=mydatetime2)

        ev_num = random.randrange(10, 100)
        if dev['dev_type'] == 'tablet':
            user_agent = "https://developers.whatismybrowser.com/useragents/parse/864604-safari-ios-ipad-webkit"
        elif dev['dev_type'] == 'desktop':
            user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"
        else:
            user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299"

        values_to_add = {
            'stime': mydatetime, 
            'etime': mydatetime2, 
            'src_ip': myip, 
            'user_agent': user_agent, 
            'dev_type': dev['dev_type']
            }
        row_to_add = pd.Series(values_to_add, name = j)
        df_event_summary = df_event_summary.append(row_to_add)

In [ ]:
len(df_event_summary)

In [ ]:
# Write out the DHCP Log
df_dhcp_sorted = df_dhcp.sort_values('time')
DHCP_LOG = []
i = 0
for index, row in df_dhcp_sorted.iterrows():
    i = i + 1
    new_log_entry = DHCPLog(i, row['date'], row['time'], row['ip'], row['macaddr'], 'Initiated IP Lease')
    DHCP_LOG.append(new_log_entry.stringify())
        
with open(r'dhcp.log', 'w+') as f:
    for event in DHCP_LOG:
        f.write(event)
        f.write("\n")

In [ ]:
df_event = pd.DataFrame(columns={"time", "method", "src_ip", "url", "request", "dst_ip", "user_agent"})
for index, row in df_event_summary:
    for i in range(1, random.randrange(5,100)):
        if row['dev_type'] == 'tablet':
            event = OutboundEvent(time, '', stu_endpoints)
        elif row['dev_type'] == 'desktop':
            event = OutboundEvent(time, '', endpoints)
        else:
            event = OutboundEvent(time, '', teacher_endpoints)
        values_to_add = {
            "time": faker.date_time_between(start_date=row['stime'], end_date=row['etime']), 
            "method": event.method, 
            "src_ip": row['src_ip'], 
            "url": event.url, 
            "request": event.request, 
            "dst_ip": event.dst_ip, 
            "user_agent": row['user_agent']
            }
        row_to_add = pd.Series(values_to_add, name = j)
        df_event_summary = df_event_summary.append(row_to_add)
            
print('done')

In [ ]:
# Write out web log
df_events_sorted = df_events.sort_values("time")
WEB_EVENTS = []
i = 0
for index, row in df_events_sorted.iterrows():
    i += 1
    new_event = ' '.join([str(x) for x in [row['time'], row['method'], row['src_ip'], row['url'], row['request'], row['dst_ip'], row['user_agent']]])
    WEB_LOG.append(new_event)
    
with open(r'weblog.log', 'w+') as f:
    for event in DHCP_LOG:
        f.write(event)
        f.write("\n")
        
# def stringify(self):
#     """Print the data in one convinient line"""
#     logline = ' '.join([str(x) for x in [self.time, self.method, self.src_ip, self.url, self.request, self.dst_ip,
#                                          self.user_agent]])

#     return logline

In [ ]:
def generate_web_traffic(start_dt, end_dt):
        time = faker.date_time_between(start_date='-8h', end_date=end_dt)
        new_event = OutboundEvent(time, hosts, endpoints).stringify()
        WEB_EVENTS.append(new_event)

In [ ]:
with open(r'dhcp.log', 'w+') as f:
    for event in DHCP_LOG:
        f.write(event)
        f.write("\n")

In [ ]:
print(faker.ipv4())
print(faker.uri())

In [ ]:
faker.date_time_between(start_date='-8h', end_date='now')

In [ ]:
time = datetime.